# Data preparation

In [56]:
from datasets import load_from_disk

def new_column(example):
    example["ner_tags"] = example["labels"]
    return example

# data = load_from_disk("test.hf")
data = load_from_disk("testWithOrigin.hf")
id_column = range(data.num_rows)
data = data.add_column("id", id_column)
data = data.map(new_column)

# Split up the data for testing and training
data = data.train_test_split(test_size=0.1)
test_data = data["test"]
data = data["train"].train_test_split(test_size=0.2)

print(data, test_data)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'origin', 'id', 'ner_tags'],
        num_rows: 304
    })
    test: Dataset({
        features: ['tokens', 'labels', 'origin', 'id', 'ner_tags'],
        num_rows: 76
    })
}) Dataset({
    features: ['tokens', 'labels', 'origin', 'id', 'ner_tags'],
    num_rows: 43
})


In [57]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

example = data["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]', 'somebody', 'catch', 'that', 'man', '!', '[SEP]']

In [58]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_data = data.map(tokenize_and_align_labels, batched=True)
tokenized_data["train"][0]

Map:   0%|          | 0/304 [00:00<?, ? examples/s]

Map:   0%|          | 0/76 [00:00<?, ? examples/s]

{'tokens': ['Somebody', 'catch', 'that', 'man', '!'],
 'labels': [-100, 1, 0, 4, 4, 0, -100],
 'origin': [1.1, None, 1.0, 1.0, 0.0],
 'id': 391,
 'ner_tags': [1, 0, 4, 4, 0],
 'input_ids': [101, 8307, 4608, 2008, 2158, 999, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [59]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Evaluation method

In [60]:
import evaluate
import numpy as np

seqeval = evaluate.load("seqeval")

label_list = [
    "O",
    "Agent",
    "Location",
    "Patient",
    "Theme",
    "Topic",
    "Destination",
    "Result",
]

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Training

In [61]:
id2label = {
    0: "O",
    1: "Agent",
    2: "Location",
    3: "Patient",
    4: "Theme",
    5: "Topic",
    6: "Destination",
    7: "Result",
}

label2id = {
    "O": 0,
    "Agent": 1,
    "Location": 2,
    "Patient": 3,
    "Theme": 4,
    "Topic": 5,
    "Destination": 6,
    "Result": 7,
}

In [62]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=8, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
# Training parameters
training_args = TrainingArguments(
    output_dir="thematic_role_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=50,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.811764,0.000000,0.000000,0.000000,0.777778
2,No log,0.682679,0.000000,0.000000,0.000000,0.779693
3,No log,0.551691,0.358491,0.246753,0.292308,0.787356
4,No log,0.462175,0.485714,0.441558,0.462585,0.831418
5,No log,0.414436,0.602941,0.532468,0.565517,0.854406
6,No log,0.399662,0.577778,0.675325,0.622754,0.873563
7,No log,0.392761,0.571429,0.623377,0.596273,0.873563
8,No log,0.412462,0.559524,0.610390,0.583851,0.873563
9,No log,0.441488,0.577778,0.675325,0.622754,0.881226
10,No log,0.458417,0.588889,0.688312,0.634731,0.886973


C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Theme seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Agent seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Location seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Result seems not to be NE tag.
  warnings.w

C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Theme seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Agent seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Location seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Result seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Patient seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\met

C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Theme seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Agent seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Location seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Result seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Patient seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\bikow\anaconda3\Lib\site-packages\seqeval\met

RuntimeError: [enforce fail at inline_container.cc:424] . unexpected pos 298959360 vs 298959248

# Inference

In [41]:
from transformers import pipeline

text = "A dog chases a cat to the bus."

classifier = pipeline("ner", model="thematic_role_model/checkpoint-506")
classifier(text)

[{'entity': 'Agent',
  'score': 0.9974968,
  'index': 1,
  'word': 'a',
  'start': 0,
  'end': 1},
 {'entity': 'Agent',
  'score': 0.99777067,
  'index': 2,
  'word': 'dog',
  'start': 2,
  'end': 5},
 {'entity': 'Theme',
  'score': 0.9660569,
  'index': 4,
  'word': 'a',
  'start': 13,
  'end': 14},
 {'entity': 'Theme',
  'score': 0.9869752,
  'index': 5,
  'word': 'cat',
  'start': 15,
  'end': 18},
 {'entity': 'Destination',
  'score': 0.9916328,
  'index': 7,
  'word': 'the',
  'start': 22,
  'end': 25},
 {'entity': 'Destination',
  'score': 0.99160063,
  'index': 8,
  'word': 'bus',
  'start': 26,
  'end': 29}]

In [63]:
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("thematic_role_model/checkpoint-110")
inputs = tokenizer(text, return_tensors="pt")

from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("thematic_role_model/checkpoint-110")
with torch.no_grad():
    logits = model(**inputs).logits

predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]

print(text)
print(inputs)
print(predicted_token_class)

A dog chases a cat to the bus.
{'input_ids': tensor([[  101,  1037,  3899, 29515,  1037,  4937,  2000,  1996,  3902,  1012,
           102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
['O', 'Agent', 'Agent', 'O', 'Theme', 'Theme', 'O', 'Destination', 'Destination', 'O', 'O']
